## Use tesseract or easyocr to detect text only:

1) Calculate the coordinates where the text's are located in the image.
2) Draw bounding boxes on these coordinates.
3) Extract only the texts in these bounding boxes.

In [ ]:
# Pip installation commands

In [ ]:
# Import statements

In [ ]:
# Image pre-processing

In [ ]:
# Text extraction method

In [ ]:
# Method call